In [2]:
import pandas as pd
import numpy as np
import torch
from pathlib import Path
import json
import os
import collections
import re 
from imageio import imread, imwrite
from multiprocessing import Pool
import matplotlib.pyplot as plt
import cv2
from scipy import linalg
#import matplotlib.pyplot as plt

/usrhomes/g013/anaconda3/envs/myenv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## FSD Calculation

In [ ]:
## The format used in semantic pallete is prob BxClassXHXW - > They pass to the function in the end BxClassMeans

In [ ]:
# One Way to do it .. convert ur tensor into 1 hot encoding, then calculate mean over that.

In [68]:
temp= torch.zeros(1,2,3,4)
res= torch.mean(d_sem, dim=(2, 3))

In [69]:
res.shape

torch.Size([1, 2])

In [144]:
path = "data/aachen_000056_000019_gtFine_labelIds.png"
sem_seg = imread(path)
sem_seg_tensor = torch.tensor(sem_seg)


/tmp/ipykernel_3170217/50100810.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  sem_seg = imread(path)
/tmp/ipykernel_3170217/50100810.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  sem_seg_2 = imread(path2)


In [162]:
# Calculate mean value of 1 hot classes for 1 image
sem_seg_squeezed=sem_seg_tensor.reshape(-1)
res=torch.nn.functional.one_hot(sem_seg_squeezed.to(torch.int64), -1) 
final_res = res.reshape(sem_seg_tensor.shape[0],sem_seg_tensor.shape[1],-1).float()
mean_values = torch.mean(final_res, dim=(0, 1))
#append
#Concatenate

In [163]:
# arr = [mean_values.cpu().numpy(),mean_values.cpu().numpy()]
# hoba = np.concatenate(arr)

In [164]:
hoba.shape

(54,)

array(0.00676835)

## FSD Caluculation on Images

#### Frechet distance Calculation

In [117]:

def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):


    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    assert mu1.shape == mu2.shape, \
        'Training and test mean vectors have different lengths'
    assert sigma1.shape == sigma2.shape, \
        'Training and test covariances have different dimensions'

    diff = mu1 - mu2

    # Product might be almost singular
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        # msg = ('fid calculation produces singular product; '
        #        'adding %s to diagonal of cov estimates') % eps
        # print(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    # Numerical error might give slight imaginary component
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError('Imaginary component {}'.format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return (diff.dot(diff) + np.trace(sigma1) +
            np.trace(sigma2) - 2 * tr_covmean)


In [118]:
def calculate_fsd(real_data,generated_data):
    """Calculates the FSD of two paths"""
    m1 = np.mean(real_data, axis=0)
    s1 = np.cov(real_data, rowvar=False)

    m2 = np.mean(generated_data, axis=0)
    s2 = np.cov(generated_data, rowvar=False)
    fsd_value = calculate_frechet_distance(m1, s1, m2, s2)
    print(f"FSD value is {fsd_value}")
    return fsd_value

#### Calculate Mean and Cov of Dataset

In [119]:
dataset_path= "/no_backups/g013/data/v_2"


In [122]:
dataset_mean_values=[]
accum=0
classes=16
for subdir, _ , files in os.walk(dataset_path):
    for file in files:
        accum+=1
        if accum%10 ==0:
            print(f"Done with {(accum/5000)*100}% of the data")
        if "labelIds" not in file:
            continue
        filepath = subdir + os.sep + file
        image = imread(filepath)
        sem_seg_tensor = torch.tensor(image)
        sem_seg_squeezed=sem_seg_tensor.reshape(-1)
        res=torch.nn.functional.one_hot(sem_seg_squeezed.to(torch.int64), 16) 
        final_res = res.reshape(sem_seg_tensor.shape[0],sem_seg_tensor.shape[1],-1).float()
        mean_val = torch.mean(final_res, dim=(0, 1))
        dataset_mean_values.append(mean_val.cpu().numpy())
real_dataset_cond = np.concatenate(dataset_mean_values)
        

/tmp/ipykernel_3270462/3372996444.py:12: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imread(filepath)


torch.Size([1024, 2048])
torch.Size([1024, 2048])
torch.Size([1024, 2048])
torch.Size([1024, 2048])
torch.Size([1024, 2048])
torch.Size([1024, 2048])
torch.Size([1024, 2048])
torch.Size([1024, 2048])
torch.Size([1024, 2048])
Done with 0.2% of the data
torch.Size([1024, 2048])
torch.Size([1024, 2048])
torch.Size([1024, 2048])
torch.Size([1024, 2048])
torch.Size([1024, 2048])
torch.Size([1024, 2048])
torch.Size([1024, 2048])
torch.Size([1024, 2048])
torch.Size([1024, 2048])
torch.Size([1024, 2048])
Done with 0.4% of the data
torch.Size([1024, 2048])
torch.Size([1024, 2048])
torch.Size([1024, 2048])
torch.Size([1024, 2048])
torch.Size([1024, 2048])


KeyboardInterrupt: 

In [187]:
real_dataset_cond

array([0.05396271, 0.30766678, 0.1336875 , ..., 1.        , 1.        ,
       1.        ], dtype=float32)

#### Calculate mean and covariance of model

In [179]:
import sys
sys.path.insert(0,'../SemanticStyleGAN')
from models import make_model
from visualize.utils import generate

OSError: CUDA_HOME environment variable is not set. Please set it to your CUDA install root.

In [1]:
ckpt="/no_backups/g013/checkpoints/version_3/ckpt/045000.pt"

In [184]:
def initalize_model(ckpt,device):
    ckpt = torch.load(ckpt)
    model = make_model(ckpt['args'])
    model.to(device)
    model.eval()
    model.load_state_dict(ckpt['g_ema'])


In [124]:
truncation_mean=10000
truncation=0.7
sample=10000
batch=8
device="cuda"
initalize_model(ckpt,"cuda")
mean_latent = model.style(torch.randn(truncation_mean, model.style_dim, device=device)).mean(0)
start_time = time.time()
dataset_mean_values=[]
with torch.no_grad():
    styles = model.style(torch.randn(sample, model.style_dim, device=device))
    styles = truncation * styles + (1-truncation) * mean_latent.unsqueeze(0)
    images, segs = generate(model, styles, mean_latent=mean_latent, batch_size=batch)
    print(f"Time taken to generate images : {time.time()-start_time}")
    for i in range(len(images)):
        converted_seg=from_rgb_to_label(segs[i],color_map)
        sem_seg_tensor = torch.tensor(converted_seg)
        sem_seg_squeezed=sem_seg_tensor.reshape(-1)
        res=torch.nn.functional.one_hot(sem_seg_squeezed.to(torch.int64), -1) 
        final_res = res.reshape(sem_seg_tensor.shape[0],sem_seg_tensor.shape[1],-1).float()
        mean_val = torch.mean(final_res, dim=(0, 1))
        dataset_mean_values.append(mean_val.cpu().numpy()) 
    generated_dataset_cond=np.concatenate(dataset_mean_values)
print(f"Average speed: {(time.time() - start_time)/(sample)}s")

NameError: name 'initalize_model' is not defined

In [201]:
#calculate_fsd(real_dataset_cond,generated_dataset_cond)
calculate_fsd(real_dataset_cond,real_dataset_cond)

FSD value is 0.0


0.0

#### Mapping Segmentation Back to their original label.

In [3]:
import imageio

In [26]:
color_map = {
    0: [0, 0, 0], #Void
    1: [128, 64,128], #Road
    2: [244, 35,232], # Side Walk
    3: [70, 70, 70], # Building
    4: [102,102,156], # Wall
    5: [190,153,153],#Fence
    6: [153,153,153],# pole
    7: [250,170, 30], # traffic light
    8: [220,220,  0], #Traffic sign
    9: [107,142, 35], # Vegitation
    10: [70,130,180], #sky
    11: [220, 20, 60], #human
    12: [255,  0,  0],#rider
    13: [ 0,  0,142],#car
    14: [ 0, 60,100],# other vehicles
    15: [ 0,  0,230],  #bike and motorcycle
    16: [116, 95, 159],
}

In [123]:
def from_rgb_to_label(image,color_map):
    color_map_sum = {}
    new_image = np.zeros((image.shape[0],image.shape[1]))
    for c in color_map:
      color_map_sum[c]=sum(color_map[c])
    image_tensor = torch.tensor(image).to(float)
    image_summed=torch.sum(image_tensor, dim=2)
    for index in color_map_sum:
        mask = image_summed==color_map_sum[index]
        new_image[mask]= index
    return new_image

In [112]:
path="data/000016_seg.png"
image = imageio.imread(path)
image.shape
image_res=from_rgb_to_label(image,color_map)
image_res=image_res.astype(int)
cv2.imwrite("data/000016_seg_decoded.png",image_res)

/tmp/ipykernel_3270462/3708616580.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(path)


True

In [113]:
img_read = imageio.imread("data/000016_seg_decoded.png")
arr=[]
for s in img_read:
    for g in s :
        arr.append(g)
set(arr)

/tmp/ipykernel_3270462/962514239.py:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img_read = imageio.imread("data/000016_seg_decoded.png")


{0, 1, 2, 3, 6, 9, 10, 11, 13, 15}

In [30]:
for s in image:
    for g in s :
        if (g==color_map[1]).all():
            print("hoba")

hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba
hoba


/tmp/ipykernel_3270462/3262669561.py:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if (g==color_map[1]).all():


AttributeError: 'bool' object has no attribute 'all'